# Data Science Course
### &nbsp; &nbsp; &nbsp; Michaël Defferrard, Sept. 2016
## Lecture 2: A Python Tour of Data Science

Goal: introduce the Python stack for Data Science. This short primer is designed as a tour around the major Python packages used for the main computational tasks of Data Science. We also aim at introducting the major Python constructs.

A Data Science Pipeline:
1. Data acquisition
    1. Importation
    1. Cleaning
    1. Exploration
1. Data exploitation
    1. Information extraction
    1. Prediction

A **motivating example**: predict whether a credit card client will default.
* Binary classification task: client will default or not ($y=1$ if yes; $y=0$ if no).
* 30'000 clients from Taiwan.
* 23 numerical & categorical explanatory variables:
    1. $x_1$: Amount of the given credit.
    2. $x_2$: Gender (1 = male; 2 = female).
    3. $x_3$: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
    4. $x_4$: Marital status (1 = married; 2 = single; 3 = others).
    5. $x_5$: Age (year).
    6. $x_6$ to $x_{11}$: History of past payment (monthly from September to April, 2005) (-1 = pay duly; 1 = payment delay for one month; ...; 9 = payment delay for nine months and above).
    7. $x_{12}$ to $x_{17}$: Amount of bill statement (monthly from September to April, 2005).
    8. $x_{18}$ to $x_{23}$: Amount of previous payment (monthly from September to April, 2005).
* [From UCI ML repository](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients).

In [ ]:
import numpy as np
import pandas as pd
import sqlalchemy
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

## 1 Data acquisition

* The world is messy: we got data from CSV, JSON, Excel, HDF5, SQL database.
* Could also be: matlab, HTML/XML, web scraping, web APIs (e.g. Twitter Firhose), noSQL databases.

In [ ]:
%%bash
ls credit_card_defaults

### 1.1 Importing from a database

[SQLAlchemy](http://www.sqlalchemy.org/) to the rescue !
* Abstraction between DBAPIs.
    * Supported databases: SQLite, Postgresql, MySQL, Oracle, MS-SQL, Firebird, Sybase and others.
* [SQL Expression Language](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html).
* [Object Relational Mapper (ORM)](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html).

TODO: show some ORM

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///credit_card_defaults/payments.sqlite', echo=False)

# Infer from existing DB.
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

# An SQL SELECT statement.
table = metadata.tables.get('payments')
op = sqlalchemy.sql.select([table])
engine.echo = True
result = engine.execute(op)
engine.echo = False

In [ ]:
# Show some lines.
for row in result.fetchmany(size=10):
    print('ID: {:2d}, payments: {}'.format(row[0], row[1:]))
result.close()

In [ ]:
# Some raw SQL.
paid = 1000
op = sqlalchemy.sql.text('SELECT payments."ID", payments."PAY6" FROM payments WHERE payments."PAY6" = {}'.format(paid))
result = engine.execute(op).fetchall()
print('{} clients paid {} in April 2005'.format(len(result), paid))

### 1.2 Merging data sources

Put some [pandas](http://pandas.pydata.org/) in our Python !
* Import / export data from / to various sources.
* Data frames manipulations: slicing, dicing, grouping.
* And many more !

In [ ]:
def get_data(folder):
    demographics = pd.read_csv(folder + 'demographics.csv', skiprows=[0], index_col=0)
    delays = pd.read_excel(folder + 'delays.xls', skiprows=[0], index_col=0)
    bills = pd.read_hdf(folder + 'bills.hdf5', 'bills')
    payments = pd.read_sql('payments', engine, index_col='ID')

    target = pd.read_json(folder + 'target.json', typ='series', orient='index')
    target = pd.DataFrame(target, columns=['DEFAULT'])

    return pd.concat([demographics, delays, bills, payments, target], axis=1)

data = get_data('credit_card_defaults/')
attributes = data.columns.tolist()

# Tansform from numerical to categorical variable.
data['SEX'] = data['SEX'].astype('category')
data['SEX'].cat.categories = ['MALE', 'FEMALE']
data['MARRIAGE'] = data['MARRIAGE'].astype('category')
data['MARRIAGE'].cat.categories = ['UNK', 'MARRIED', 'SINGLE', 'OTHERS']
data['EDUCATION'] = data['EDUCATION'].astype('category')
data['EDUCATION'].cat.categories = ['UNK', 'GRAD SCHOOL', 'UNIVERSITY', 'HIGH SCHOOL', 'OTHERS', 'UNK1', 'UNK2']

### 1.3 Looking at the data

In [ ]:
data.loc[:6, ['LIMIT', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'DEFAULT']]

In [ ]:
data.iloc[:5, 4:10]

In [ ]:
data.iloc[:5, 11:23]

Export as an [HTML table](./subset.html) for manual inspection.

In [ ]:
data[:1000].to_html('subset.html')

## 2. Data cleaning

The most boring part. But the most time intensive !

TODO: show a study. Which one was it, KDnuggets ?

Marital status
* Should be either 1 (married), 2 (single) or 3 (others).
* Let's *assume* the 0 represents errors when collecting the data and remove those clients.

In [ ]:
print(data['MARRIAGE'].value_counts())
data = data[data['MARRIAGE'] != 'UNK']
data['MARRIAGE'] = data['MARRIAGE'].cat.remove_unused_categories()
print('\nWe are left with {} clients\n'.format(data.shape))
print(data['MARRIAGE'].unique())

Education
* Should be either 1 (graduate school), 2 (university), 3 (high school) or 4 (others).
* Let's *assume* the 0, 5, 6 are dubious, but do not invalidate the data. Keep them as they may have predictive power.

In [ ]:
print(data['EDUCATION'].value_counts())
data.loc[data['EDUCATION']=='UNK1', 'EDUCATION'] = 'UNK'
data.loc[data['EDUCATION']=='UNK2', 'EDUCATION'] = 'UNK'
data['EDUCATION'] = data['EDUCATION'].cat.remove_unused_categories()
print(data['EDUCATION'].value_counts())

## 3. Data exploration

* Get descriptive statistics.
* Plot informative figures.
* Verify some intuitive correlations.

TODO: further exploration with [statsmodels](http://statsmodels.sourceforge.net/)

In [ ]:
attributes_numerical = ['LIMIT', 'AGE']
attributes_numerical.extend(attributes[11:23])
data.loc[:, attributes_numerical].describe().astype(np.int)

In [ ]:
data.loc[:, 'AGE'].plot.hist(bins=20, figsize=(15,5))
ax = data.iloc[:, 11:17].plot.box(logy=True, figsize=(15,5))

In [ ]:
percentage = data[data.DEFAULT == 1].shape[0] / data.shape[0] * 100
print('Percentage of defaults: {:.2f}%'.format(percentage))

Who's more susceptible to default, males or females ?
Statistical signifiance could be tested with [scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html).

In [ ]:
pd.crosstab(data['SEX'], data['DEFAULT'])

**Intuition**: people who pay late present a higher risk of defaulting. Let's verify ! 

In [ ]:
group = data.groupby('DELAY1').mean()
group['DEFAULT'].plot(grid=True, figsize=(15,5));

## 4 Pre-processing

Back to the [NumPy](http://www.numpy.org/) foundation.

TODO: pure Python & [Numba](http://numba.pydata.org/)

## 5 First predictive model

Feel the power of [SciPy](https://www.scipy.org/) and the underlying [NumPy](http://www.numpy.org/) !

TODO: compute a gradient with [SymPy](http://www.sympy.org/)

In [ ]:
class model(object):
    def predict(self):
        pass

## 6 More models

Tired of writing algorithms ? Try [scikit-learn](https://www.scipy.org/).

## 7 Deep Learning

Let's do it with [Keras](https://keras.io/) (and its [TensorFlow](https://www.tensorflow.org/) backend).

## 8 Visualization

TODO: interactive ([matplotlib](http://matplotlib.org/) widget, [Bokeh](http://bokeh.pydata.org))